#Data Set

In [4]:
import pandas as pd
import numpy as np
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")

xmas.head()

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,partridge,NaN,NaN,in a pear tree
1,2,second,dove,NaN,turtle,NaN
2,3,third,hen,NaN,french,NaN
3,4,fourth,bird,NaN,calling,NaN
4,5,fifth,ring,NaN,golden,NaN


#Function 1: pluralize_gift()

In [5]:
def pluralize_gift(gift):
    if "oo" in gift:
        gift = gift.replace("oo", "ee")
    elif gift[-1] == "y":
        gift = gift.replace("y", "ies")
    else:
        gift = gift + "s"
    return gift

# Vectorize 
vectorized_pluralize = np.vectorize(pluralize_gift)

#Test Your Function

In [6]:
pluralize_gift("goose")

'geese'

In [7]:
vectorized_pluralize(xmas['Gift.Item'])

array(['partridges', 'doves', 'hens', 'birds', 'rings', 'geese', 'swans',
       'maids', 'ladies', 'lords', 'pipers', 'drummers'], dtype='<U10')

#Function 2: make_phrase()

In [8]:
# Dictionary mapping day numbers to cardinal numbers
cardinal_numbers = {
    1: "one",
    2: "two",
    3: "three",
    4: "four",
    5: "five",
    6: "six",
    7: "seven",
    8: "eight",
    9: "nine",
    10: "ten",
    11: "eleven",
    12: "twelve"
}

In [9]:
def make_phrase(num, num_word, item, verb, adjective, location):
    """
    <documentation here>
    """
    
    ## Step 1: Replace NAs with blank strings
    verb = verb if pd.notna(verb) else ""
    adjective = adjective if pd.notna(adjective) else ""
    location = location if pd.notna(location) else ""
    
    ## Step 2: If the day number is larger than 1, the gift items need pluralized!
    ### Hint: call the function you created above!
    if num > 1:
        item = pluralize_gift(item)
    
    ## Step 3: Figure out if a gift item starts with a vowel
    vowels = "aeiou"
    starts_with_vowel = item[0].lower() in vowels
    
    ## Step 4: For the first day, if the gift item starts with a vowel, replace the day with "an" and if the gift item does not start with a vowel, replace the day with "a" (e.g. a partridge in a pear tree). If it is not the first day, use just the number word (e.g. ten lords a leap)
    if num > 1:
        num_word = cardinal_numbers[num]
        
    if num == 1:
        if starts_with_vowel:
            num_word = "an"
        else:
            num_word = "a"
    
    ## Step 5: Put all of the pieces together into one string and return!

    # Ensure that any blanks are skipped by checking their length
    phrase_parts = [num_word, adjective, item, verb, location]

    # Join the non-blank parts with spaces
    phrase = " ".join(part for part in phrase_parts if part)
    
    return phrase

#Test Your Function

In [10]:
make_phrase(num=1, 
num_word="one", 
item="partridge", 
verb="in a pear tree", 
adjective="", 
location="")

'a partridge in a pear tree'

In [11]:
xmas['Full.Phrase'] = xmas.apply(
    lambda row: make_phrase(
        num=row['Day'],
        num_word=row['Day.in.Words'],
        item=row['Gift.Item'],
        verb=row['Verb'],
        adjective=row['Adjective'],
        location=row['Location']
    ),
    axis=1
)

xmas

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location,Full.Phrase
0,1,first,partridge,NaN,NaN,in a pear tree,a partridge in a pear tree
1,2,second,dove,NaN,turtle,NaN,two turtle doves
2,3,third,hen,NaN,french,NaN,three french hens
3,4,fourth,bird,NaN,calling,NaN,four calling birds
4,5,fifth,ring,NaN,golden,NaN,five golden rings
5,6,sixth,goose,a-laying,NaN,NaN,six geese a-laying
6,7,seventh,swan,a-swimming,NaN,NaN,seven swans a-swimming
7,8,eighth,maid,a-milking,NaN,NaN,eight maids a-milking
8,9,ninth,lady,dancing,NaN,NaN,nine ladies dancing
9,10,tenth,lord,a-leaping,NaN,NaN,ten lords a-leaping


#Function 3: sing_day()

In [12]:
def sing_day(dataset, num, phrase_col):
    """
    Function to generate lyrics for a specific day of Christmas in the desired format.
    """

    # Step 1: Setup the intro line
    num_word = {
        1: "first",
        2: "second",
        3: "third",
        4: "fourth",
        5: "fifth",
        6: "sixth",
        7: "seventh",
        8: "eighth",
        9: "ninth",
        10: "tenth",
        11: "eleventh",
        12: "twelfth"
    }

    intro = "On the " + num_word[num] + " day of Christmas, my true love sent to me:"
  
    # Step 2: Sing the gift phrases in reverse order
    gifts = []

    # Gifts are sung in reverse order, starting from the current day down to day 1
    for i in range(num, 0, -1):
        phrase = dataset.loc[dataset['Day'] == i, phrase_col].values[0]
        gifts.append(phrase)
    
    # Step 3: Format the gift list
    if num > 1:

        # Add "and" before the last gift
        gifts[-1] = "and " + gifts[-1]  
    
    # Join gifts with commas and newlines
    gift_str = ",\n".join(gifts)  
    
    # Step 4: Put it all together and return
    return intro + "\n" + gift_str + "."

#Test your function

In [13]:
print(sing_day(xmas, 3, "Full.Phrase"))

On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.
On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.


#Use Your Functions!

In [18]:
def sing_full_song(dataset, phrase_col):

    full_song = ""
    
    # Loop through each day (from 1 to 12) to generate the song
    for day in range(1, 13):
        # Generate the lyrics for each day
        day_lyrics = sing_day(dataset, day, phrase_col)
        # Append each day's lyrics to the full song
        full_song += day_lyrics + "\n\n"
    
    return full_song

# Example of how to call it
full_song = sing_full_song(xmas, "Full.Phrase")

print(full_song)

On the first day of Christmas, my true love sent to me:
a partridge in a pear tree.

On the second day of Christmas, my true love sent to me:
two turtle doves,
and a partridge in a pear tree.

On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fourth day of Christmas, my true love sent to me:
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fifth day of Christmas, my true love sent to me:
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the sixth day of Christmas, my true love sent to me:
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the seventh day of Christmas, my true love sent to me:
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a 

In [15]:
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")

In [19]:
xmas2['Full.Phrase'] = xmas2.apply(
    lambda row: make_phrase(
        row['Day'],
        row['Day.in.Words'],
        row['Gift.Item'],
        row['Verb'],
        row['Adjective'],
        row['Location']
    ),
    axis=1
)

In [20]:
# Printing Full Song
print(sing_full_song(xmas2, 'Full.Phrase'))

On the first day of Christmas, my true love sent to me:
an email from Cal Poly.

On the second day of Christmas, my true love sent to me:
two meal points,
and an email from Cal Poly.

On the third day of Christmas, my true love sent to me:
three lost pens,
two meal points,
and an email from Cal Poly.

On the fourth day of Christmas, my true love sent to me:
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the fifth day of Christmas, my true love sent to me:
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the sixth day of Christmas, my true love sent to me:
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the seventh day of Christmas, my true love sent to me:
seven seniors stressing,
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On t